In [1]:
import os
import sys
import glob
import time
import copy
import logging
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils
import torch.nn.functional as F
import model_utils.configure as conf
from model import Graph2Seq
import utils

import math
import json
from collections import OrderedDict

import numpy as np
import torch
import torch.utils.data
import torch.nn.functional as F

INPUT = 'input'
CONV1X1 = 'conv1x1-bn-relu'
CONV3X3 = 'conv3x3-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
OUTPUT = 'output'


In [2]:
mode = "train"
random.seed(conf.seed)
np.random.seed(conf.seed)
torch.manual_seed(conf.seed)
logging.info("conf = %s", conf)

conf.source_length = conf.encoder_length = conf.decoder_length = (conf.nodes + 2) * (conf.nodes - 1) // 2
epochs = conf.epochs

# model = Graph2Seq(mode=mode, conf=conf)

# load data
#dataset = utils.ControllerDataset(conf.data_file_path, conf.graph_size)
#queue = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, pin_memory=False)
# queue = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, pin_memory=True)

In [4]:
# del dataset
# data_file_path = "data/train_data.json"
# dataset = utils.ControllerDataset(conf.data_file_path)
queue = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, pin_memory=False, collate_fn=utils.collate_fn)

In [5]:
def train_step(train_queue, optimizer):
    objs = utils.AvgrageMeter()
    nll = utils.AvgrageMeter()
    print("train_queue {}".format(train_queue))
    for step, sample in enumerate(train_queue):
        fw_adjs = sample['fw_adjs']
        bw_adjs = sample['bw_adjs']
        operations = sample['operations']
        sequence = sample['sequence']

        optimizer.zero_grad()
        log_prob, predicted_value = model(fw_adjs, bw_adjs, operations, targets=sequence)
        loss = F.nll_loss(log_prob.contiguous().view(-1, log_prob.size(-1)), sequence.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), conf.grad_bound)
        optimizer.step()
        
        n = sequence.size(0)
        objs.update(loss.data, n)
        nll.update(loss.data, n)

    return objs.avg, nll.avg


In [ ]:
model.train()

logging.info('Train data: {}'.format(len(queue)))
optimizer = torch.optim.Adam(model.parameters(), lr=conf.lr, weight_decay=conf.l2_reg)

# for epoch in range(1, epochs + 1):
for epoch in range(2):
    loss, ce = train_step(queue, optimizer)
    print("epoch %04d train loss %.6f ce %.6f", epoch, loss, ce)
    # logging.info("epoch %04d train loss %.6f ce %.6f", epoch, loss, ce)

## save trainable parameters
torch.save(model.state_dict(), conf.model_path)

In [6]:
import logging
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from model_utils.aggregators import MeanAggregator
import model_utils.configure as conf
import utils

from decoder import Decoder
from encoder import Encoder



In [ ]:
class Graph2Seq(nn.Module):

    def __init__(self, mode, conf):
        super(Graph2Seq, self).__init__()

        self.mode = mode
        self.l2_lambda = conf.l2_lambda
        self.feature_embedding_dim = conf.hidden_layer_dim
        self.vocab_size = conf.vocab_size
        self.length = conf.vocab_size -1
        self.hop_size = conf.hop_size

        # the setting for the GCN
        self.graph_encode_direction = conf.graph_encode_direction
        self.concat = conf.concat

        # the setting for the decoder
        self.single_graph_nodes_size = conf.graph_size
        self.attention = conf.attention
        self.decoder_layers = conf.decoder_layers

        self.dropout = conf.dropout
        self.learning_rate = conf.learning_rate

        self.if_pred_on_dev = False

        self.encoder = Encoder(
                                mode=mode,
                                vocab_size=self.vocab_size,
                                feature_embedding_dim=self.feature_embedding_dim,
                                graph_encode_direction=self.graph_encode_direction,
                                hop_size=self.hop_size,
                                concat=self.concat,
                                dropout=self.dropout,
                                learning_rate=self.learning_rate
                                )

        self.decoder = Decoder(
                                mode=mode,
                                hidden_dim=self.feature_embedding_dim,
                                vocab_size=self.vocab_size,
                                dropout=self.dropout,
                                length=self.length,
                                layers=self.decoder_layers
                                )


        """
        encoder, decoder가 공유해야하는 것 : node feature embedding -> should it?
        encoder에서는 operation이 ndoe feature
        decoder에서는 operation + connection 이 node feature
        seminas에서는 다른 embedding 이용
        graph2sec 에서는 동일한 embedding 이용
        나는 다르게 써야겠당
        """

    def forward(self, fw_adjs, bw_adjs, operations, targets=None):
        encoded_nodes, graph_embedding = self.encoder(fw_adjs, bw_adjs, operations)
        # check dimension
        decoder_input = torch.cat([graph_embedding, targets], dim=1)
        predicted_softmax, decoded_ids = self.decoder(graph_embedding, targets=targets)

        return predicted_softmax, decoded_ids

In [ ]:
class Graph2Seq(nn.Module):

    def __init__(self, mode, conf):
        super(Graph2Seq, self).__init__()

        self.mode = mode
        self.l2_lambda = conf.l2_lambda
        self.feature_embedding_dim = conf.hidden_layer_dim
        self.vocab_size = conf.vocab_size
        self.length = conf.vocab_size -1
        self.hop_size = conf.hop_size

        # the setting for the GCN
        self.graph_encode_direction = conf.graph_encode_direction
        self.concat = conf.concat

        # the setting for the decoder
        self.single_graph_nodes_size = conf.graph_size
        self.attention = conf.attention
        self.decoder_layers = conf.decoder_layers

        self.dropout = conf.dropout
        self.learning_rate = conf.learning_rate

        self.if_pred_on_dev = False

        self.encoder = Encoder(
                                mode=mode,
                                vocab_size=self.vocab_size,
                                feature_embedding_dim=self.feature_embedding_dim,
                                graph_encode_direction=self.graph_encode_direction,
                                hop_size=self.hop_size,
                                concat=self.concat,
                                dropout=self.dropout,
                                learning_rate=self.learning_rate
                                )

        self.decoder = Decoder(
                                mode=mode,
                                hidden_dim=self.feature_embedding_dim,
                                vocab_size=self.vocab_size,
                                dropout=self.dropout,
                                length=self.length,
                                layers=self.decoder_layers
                                )


        """
        encoder, decoder가 공유해야하는 것 : node feature embedding -> should it?
        encoder에서는 operation이 ndoe feature
        decoder에서는 operation + connection 이 node feature
        seminas에서는 다른 embedding 이용
        graph2sec 에서는 동일한 embedding 이용
        나는 다르게 써야겠당
        """

    def forward(self, fw_adjs, bw_adjs, operations, targets=None):
        encoded_nodes, graph_embedding = self.encoder(fw_adjs, bw_adjs, operations)
        # check dimension
        decoder_input = torch.cat([graph_embedding, targets], dim=1)
        predicted_softmax, decoded_ids = self.decoder(graph_embedding, targets=targets)

        return predicted_softmax, decoded_ids